In [1]:
from models.self_supervised_caffe import convert_caffe_to_pytorch

In [2]:
net = convert_caffe_to_pytorch(gpu=0)

/users/ruthfong/anaconda3/envs/caffe_py27/lib/python2.7/site-packages/skimage/io/_io.py:49: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')
/users/ruthfong/anaconda3/envs/caffe_py27/lib/python2.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/users/ruthfong/anaconda3/envs/caffe_py27/lib/python2.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


> /home/ruthfong/pytorch_models/models/self_supervised_caffe.py(26)convert_caffe_to_pytorch()
-> return net
(Pdb) c


In [13]:
import os

from caffe.proto import caffe_pb2
import google.protobuf.text_format as txtf

import torch
import torch.nn as nn

from models.self_supervised_caffe import DEFAULT_CAFFE_MODELS_DIR
from models.self_supervised import PUZZLE

task_name = PUZZLE
prototxt_path = os.path.join(DEFAULT_CAFFE_MODELS_DIR, task_name + '.prototxt')

net_p = caffe_pb2.NetParameter()

with open(prototxt_path) as f:
    s = f.read()
    txtf.Merge(s, net_p)

layer_names = [l.name for l in net_p.layer]

In [5]:
layer_names

[u'conv1_s1',
 u'relu1_s1',
 u'pool1_s1',
 u'norm1_s1',
 u'conv2_s1',
 u'relu2_s1',
 u'pool2_s1',
 u'norm2_s1',
 u'conv3_s1',
 u'relu3_s1',
 u'conv4_s1',
 u'relu4_s1',
 u'conv5_s1',
 u'relu5_s1',
 u'pool5']

In [83]:
net_p.layer[3]

name: "norm1_s1"
type: "LRN"
bottom: "pool1_s1"
top: "norm1_s1"
lrn_param {
  local_size: 5
  alpha: 9.99999974738e-05
  beta: 0.75
}

In [105]:
new_model = nn.Sequential(*[
    nn.Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4)),
nn.ReLU(inplace=True),
nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True),
nn.LocalResponseNorm(5, alpha=9.99999974738e-05, beta=0.75, k=1.0),
nn.Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2),
nn.ReLU(inplace=True),
nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True),
nn.LocalResponseNorm(5, alpha=9.99999974738e-05, beta=0.75, k=1.0),
nn.Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
nn.ReLU(inplace=True),
nn.Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2),
nn.ReLU(inplace=True),
nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2),
nn.ReLU(inplace=True),
nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True),
])

In [114]:
'torch.nn' in str(type(m))

True

In [118]:
assert isinstance(features, nn.Sequential)
str_rep = 'Sequential(*[\n'
for m in features:
    prefix = 'nn.' if 'torch.nn' in repr(type(m)) else ''
    str_rep += prefix + str(m) + ',\n'
str_rep += '])'
print(str_rep)

Sequential(*[
nn.Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4)),
nn.ReLU(inplace=True),
nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True),
nn.LocalResponseNorm(5, alpha=9.99999974738e-05, beta=0.75, k=1.0),
nn.Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2),
nn.ReLU(inplace=True),
nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True),
nn.LocalResponseNorm(5, alpha=9.99999974738e-05, beta=0.75, k=1.0),
nn.Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
nn.ReLU(inplace=True),
nn.Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2),
nn.ReLU(inplace=True),
nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2),
nn.ReLU(inplace=True),
nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True),
])


In [127]:
net_p.layer[1].top

[u'conv1_s1']

In [129]:
POOL_MAX = 0;
POOL_AVE = 1;
POOL_STOCHASTIC = 2;

def extract_param(param, default_value=None):
    if isinstance(param, int) or isinstance(param, float):
        return param
    if len(param) == 1:
        return int(param[0])
    elif len(param) == 0:
        if default_value is not None:
            return default_value
        else:
            # TODO(ruthfong): raise specific error.
            assert False
    else:
        raise NotImplementedError

get_stride = lambda p: extract_param(p.stride, 1)
get_padding = lambda p: extract_param(p.pad, 0)
get_kernel_size = lambda p: extract_param(p.kernel_size)
get_bias = lambda p: extract_param(p.bias_term, True)
get_groups = lambda p: extract_param(p.group, 1)
get_local_size = lambda p: extract_param(p.local_size, 5)
get_alpha = lambda p: extract_param(p.alpha, 1.)
get_beta = lambda p: extract_param(p.beta, 0.75)
get_k = lambda p: extract_param(p.k, 1.)

prev_dim = 3
inplace = True
new_layers = []
learnable_layers = {}

for i, layer in enumerate(net_p.layer):
    if layer.type == 'Convolution':
        p = layer.convolution_param
        kernel_size = get_kernel_size(p)
        stride = get_stride(p)
        padding = get_padding(p)
        bias = get_bias(p)
        groups = get_groups(p)
        new_layer = nn.Conv2d(prev_dim,
                              p.num_output,
                              kernel_size,
                              stride=stride,
                              padding=padding,
                              dilation=1,
                              groups=groups,
                              bias=bias,
                              padding_mode='zeros')
        assert len(layer.top) == 1
        learnable_layers[layer.top[0]] = i
        prev_dim = p.num_output
    elif layer.type == 'ReLU':
        assert len(layer.top) == 1
        assert len(layer.bottom) == 1
        new_layer = nn.ReLU(inplace=inplace if inplace else layer.top[0] == layer.bottom[0])
    elif layer.type == 'Pooling':
        p = layer.pooling_param
        kernel_size = get_kernel_size(p)
        stride = get_stride(p)
        padding = get_padding(p)
        ceil_mode = True  # caffe uses ceil_mode=True
        if p.pool is POOL_MAX:
            new_layer = nn.MaxPool2d(kernel_size,
                                     stride=stride,
                                     padding=padding,
                                     dilation=1,
                                     return_indices=False,
                                     ceil_mode=ceil_mode)
        else:
            raise NotImplementedError
    elif layer.type == 'LRN':
        p = layer.lrn_param
        local_size = get_local_size(p)
        alpha = get_alpha(p)
        beta = get_beta(p)
        k = get_k(p)
        new_layer = nn.LocalResponseNorm(local_size,
                                         alpha=alpha,
                                         beta=beta,
                                         k=k)
    else:
        raise NotImplementedError
    print(i, layer)#, new_layer)
    new_layers.append(new_layer)

(0, name: "conv1_s1"
type: "Convolution"
bottom: "data"
top: "conv1_s1"
convolution_param {
  num_output: 96
  kernel_size: 11
  stride: 4
}
)
(1, name: "relu1_s1"
type: "ReLU"
bottom: "conv1_s1"
top: "conv1_s1"
)
(2, name: "pool1_s1"
type: "Pooling"
bottom: "conv1_s1"
top: "pool1_s1"
pooling_param {
  pool: MAX
  kernel_size: 3
  stride: 2
}
)
(3, name: "norm1_s1"
type: "LRN"
bottom: "pool1_s1"
top: "norm1_s1"
lrn_param {
  local_size: 5
  alpha: 9.99999974738e-05
  beta: 0.75
}
)
(4, name: "conv2_s1"
type: "Convolution"
bottom: "norm1_s1"
top: "conv2_s1"
convolution_param {
  num_output: 256
  pad: 2
  kernel_size: 5
  group: 2
}
)
(5, name: "relu2_s1"
type: "ReLU"
bottom: "conv2_s1"
top: "conv2_s1"
)
(6, name: "pool2_s1"
type: "Pooling"
bottom: "conv2_s1"
top: "pool2_s1"
pooling_param {
  pool: MAX
  kernel_size: 3
  stride: 2
}
)
(7, name: "norm2_s1"
type: "LRN"
bottom: "pool2_s1"
top: "norm2_s1"
lrn_param {
  local_size: 5
  alpha: 9.99999974738e-05
  beta: 0.75
}
)
(8, name: "con

In [135]:
features = nn.Sequential(*new_layers)
for param_name, param in net.params.items():
    layer_i = learnable_layers[param_name]
    num_params = len(param)
    assert num_params > 0
    assert num_params < 3
    assert 'conv' in param_name
    features[layer_i].weight.data[...] = torch.from_numpy(param[0].data)
    if num_params == 2:
        features[layer_i].bias.data[...] = torch.from_numpy(param[1].data)

In [21]:
model = models.alexnet()

In [176]:
from torchvision.models.utils import load_state_dict_from_url

model_urls = {
    'weakly_solvingpuzzle': 'https://github.com/ruthcfong/pytorch_models/releases/download/v2.0/alexnet-weakly_solvingpuzzle-7e75c18e.pt',
}

class AlexNetPuzzle(nn.Module):
    def __init__(self, params=None):
        super(AlexNetPuzzle, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75),
            nn.Conv2d(96, 256, kernel_size=5, groups=2, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75),
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, groups=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, groups=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        if params:
            self.load_caffe_weights(params)

    def forward(self, x):
        return self.features(x)

    def load_caffe_weights(self, params):
        self.features[0].weight.data[...] = torch.from_numpy(params['conv1_s1'][0].data)
        self.features[0].bias.data[...] = torch.from_numpy(params['conv1_s1'][1].data)
        self.features[4].weight.data[...] = torch.from_numpy(params['conv2_s1'][0].data)
        self.features[4].bias.data[...] = torch.from_numpy(params['conv2_s1'][1].data)
        self.features[8].weight.data[...] = torch.from_numpy(params['conv3_s1'][0].data)
        self.features[8].bias.data[...] = torch.from_numpy(params['conv3_s1'][1].data)
        self.features[10].weight.data[...] = torch.from_numpy(params['conv4_s1'][0].data)
        self.features[10].bias.data[...] = torch.from_numpy(params['conv4_s1'][1].data)
        self.features[12].weight.data[...] = torch.from_numpy(params['conv5_s1'][0].data)
        self.features[12].bias.data[...] = torch.from_numpy(params['conv5_s1'][1].data)

def alexnet(task_name='weakly_solvingpuzzle',
            pretrained=False,
            progress=True,
            **kwargs):
    assert task_name == 'weakly_solvingpuzzle'
    if task_name == 'weakly_solvingpuzzle':
        AlexNetPuzzle(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[task_name],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

In [177]:
model = alexnet(pretrained=True)

Downloading: "https://github.com/ruthcfong/pytorch_models/releases/download/v2.0/alexnet-weakly_solvingpuzzle-7e75c18e.pt" to /users/ruthfong/.cache/torch/checkpoints/alexnet-weakly_solvingpuzzle-7e75c18e.pt
100.0%


In [158]:
new_model = nn.Sequential(
    nn.Conv2d(3, 96, kernel_size=11, stride=4),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75),
    nn.Conv2d(96, 256, kernel_size=5, groups=2, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75),
    nn.Conv2d(256, 384, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(384, 384, kernel_size=3, groups=2, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(384, 256, kernel_size=3, groups=2, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2)
)

In [170]:
model = AlexNetPuzzle(params=net.params)

In [173]:
pytorch_models_dir = '/scratch/local/ssd/ruthfong/models/net_dissect/pytorch'
torch.save(model.state_dict(), os.path.join(pytorch_models_dir, caffe_model_name + '.pt'))

In [159]:
new_model[0].weight.data[...] = torch.from_numpy(net.params['conv1_s1'][0].data)
new_model[0].bias.data[...] = torch.from_numpy(net.params['conv1_s1'][1].data)
new_model[4].weight.data[...] = torch.from_numpy(net.params['conv2_s1'][0].data)
new_model[4].bias.data[...] = torch.from_numpy(net.params['conv2_s1'][1].data)
new_model[8].weight.data[...] = torch.from_numpy(net.params['conv3_s1'][0].data)
new_model[8].bias.data[...] = torch.from_numpy(net.params['conv3_s1'][1].data)
new_model[10].weight.data[...] = torch.from_numpy(net.params['conv4_s1'][0].data)
new_model[10].bias.data[...] = torch.from_numpy(net.params['conv4_s1'][1].data)
new_model[12].weight.data[...] = torch.from_numpy(net.params['conv5_s1'][0].data)
new_model[12].bias.data[...] = torch.from_numpy(net.params['conv5_s1'][1].data)

In [136]:
x = torch.from_numpy(net.blobs['data'].data)

In [138]:
(features(x).cpu().data.numpy() - net.blobs['pool5'].data).max()

1.23381615e-05

In [101]:
net.blobs['norm1_s1'].data.shape

(1, 96, 27, 27)

In [127]:
new_model(x).cpu().data.numpy().shape

(1, 256, 23, 23)

In [140]:
res = net.forward()

In [145]:
res.values()

[array([[[[0.        , 0.        , 0.        , 0.85551125, 1.3025935 ,
           1.3025935 ],
          [3.3760371 , 1.4088893 , 0.366484  , 0.        , 0.        ,
           0.        ],
          [4.459055  , 2.8510556 , 1.6761396 , 0.        , 0.        ,
           0.        ],
          [4.459055  , 2.8510556 , 1.0216339 , 0.        , 0.        ,
           0.        ],
          [3.5553567 , 0.        , 0.        , 0.        , 0.        ,
           0.2769971 ],
          [7.378141  , 0.5810433 , 0.72360927, 0.96689403, 1.3337975 ,
           5.6620555 ]],
 
         [[3.3095646 , 2.8439968 , 1.460573  , 0.        , 1.6661899 ,
           4.248808  ],
          [1.3726615 , 1.0828011 , 0.53819937, 3.356305  , 1.675411  ,
           8.006325  ],
          [1.7812512 , 1.7812512 , 1.279727  , 3.356305  , 1.6539552 ,
           7.875453  ],
          [1.8288398 , 1.391819  , 1.279727  , 2.2421281 , 0.        ,
           4.818714  ],
          [1.5117986 , 0.08267725, 0.85451657, 

In [144]:
res

{'pool5': array([[[[0.        , 0.        , 0.        , 0.85551125, 1.3025935 ,
           1.3025935 ],
          [3.3760371 , 1.4088893 , 0.366484  , 0.        , 0.        ,
           0.        ],
          [4.459055  , 2.8510556 , 1.6761396 , 0.        , 0.        ,
           0.        ],
          [4.459055  , 2.8510556 , 1.0216339 , 0.        , 0.        ,
           0.        ],
          [3.5553567 , 0.        , 0.        , 0.        , 0.        ,
           0.2769971 ],
          [7.378141  , 0.5810433 , 0.72360927, 0.96689403, 1.3337975 ,
           5.6620555 ]],
 
         [[3.3095646 , 2.8439968 , 1.460573  , 0.        , 1.6661899 ,
           4.248808  ],
          [1.3726615 , 1.0828011 , 0.53819937, 3.356305  , 1.675411  ,
           8.006325  ],
          [1.7812512 , 1.7812512 , 1.279727  , 3.356305  , 1.6539552 ,
           7.875453  ],
          [1.8288398 , 1.391819  , 1.279727  , 2.2421281 , 0.        ,
           4.818714  ],
          [1.5117986 , 0.08267725, 0.8